## Scenario
You're a junior data analyst at a retail company. The sales team has provided you with a CSV file containing transaction records for the past year. Your manager needs insights to make strategic decisions about inventory, regional focus, and product mix.

In [100]:
import csv
from pprint import pprint


In [101]:

STORE_DATA_FILE = "Files/sample_superstore.csv"

### Part 1 : Data Loading & Exploration

In [102]:
def load_sales_data(filename):
    """Load and clean sales data from CSV file"""
    
    sales_data = []
    
    try:
        with open(filename, "r") as file:
            reader = csv.DictReader(file)
            
            for row in reader:
                try:
                    # Convert numeric fields
                    sales = float(row['Sales'])
                    quantity = int(row['Quantity'])
                    discount = float(row['Discount'])
                    profit = float(row['Profit'])
                    
                    # Fix discount if entered as 20 instead of 0.20
                    if discount > 1:
                        discount = discount / 100
                    
                    if sales < 0:
                        continue   # skip negative sales
                    
                    if quantity <= 0:
                        continue   # skip zero or negative quantity
                    
                    if discount < 0 or discount > 1:
                        continue   # skip invalid discount
                    
                    # Save cleaned values back into row
                    row['Sales'] = sales
                    row['Quantity'] = quantity
                    row['Discount'] = discount
                    row['Profit'] = profit
                    
                    sales_data.append(row)
                
                except ValueError:
                    # Skip rows with invalid numbers
                    continue
        
        return sales_data
    
    except FileNotFoundError:
        print(f"Error: {filename} not found")
        return []
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return []
    
data = load_sales_data(STORE_DATA_FILE)
pprint("Rows loaded: " + str(len(data)))
print("************************")
pprint(data[:5])

'Rows loaded: 9994'
************************
[{'Category': 'Furniture',
  'City': 'Henderson',
  'Country': 'United States',
  'Customer ID': 'CG-12520',
  'Customer Name': 'Claire Gute',
  'Discount': 0.0,
  'Order Date': '11/08/2016',
  'Order ID': 'CA-2016-152156',
  'Postal Code': '42420',
  'Product ID': 'FUR-BO-10001798',
  'Product Name': 'Bush Somerset Collection Bookcase',
  'Profit': 41.9136,
  'Quantity': 2,
  'Region': 'South',
  'Row ID': '1',
  'Sales': 261.96,
  'Segment': 'Consumer',
  'Ship Date': '11/11/2016',
  'Ship Mode': 'Second Class',
  'State': 'Kentucky',
  'Sub-Category': 'Bookcases'},
 {'Category': 'Furniture',
  'City': 'Henderson',
  'Country': 'United States',
  'Customer ID': 'CG-12520',
  'Customer Name': 'Claire Gute',
  'Discount': 0.0,
  'Order Date': '11/08/2016',
  'Order ID': 'CA-2016-152156',
  'Postal Code': '42420',
  'Product ID': 'FUR-CH-10000454',
  'Product Name': 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back',
  'Profit': 21

In [103]:
def explore_data(sales_data):
    """Display dataset exploration statistics"""
    # Total number of orders (rows)
    total_orders = len(sales_data)
    
    # Date range
    order_dates = []
    for row in sales_data:
        order_dates.append(row["Order Date"])
    
    earliest_date = min(order_dates)
    latest_date = max(order_dates)
    
    # Unique Regions
    regions = set()
    for row in sales_data:
        regions.add(row["Region"])
    
    # Unique Categories
    categories = set()
    for row in sales_data:
        categories.add(row["Category"])
    
    # Unique products
    products = set()
    for row in sales_data:
        products.add(row["Product Name"])
    
  
    # Display results
    print("DATA EXPLORATION")
    print("-----------------")
    print("Total Orders:", total_orders)
    print("Date Range:", earliest_date, "to", latest_date)
    print("Unique Regions:", list(regions))
    print("Unique Categories:", list(categories))
    print("Number of Unique Products:", len(products))

explore_data(data)

DATA EXPLORATION
-----------------
Total Orders: 9994
Date Range: 01/01/2017 to 9/30/2017
Unique Regions: ['South', 'West', 'Central', 'East']
Unique Categories: ['Office Supplies', 'Furniture', 'Technology']
Number of Unique Products: 1850


### Part 2: Sales Performance Analysis

In [ ]:
def calculate_revenue_metrics(sales_data):
    """Calculate overall revenue metrics"""
    total_revenue = 0
    total_profit = 0
    total_quantity = 0
    
    total_revenue = sum(row["Sales"] for row in sales_data)
    total_profit = sum(row["Profit"] for row in sales_data)
    total_quantity = sum(row["Quantity"] for row in sales_data)
    
    number_of_orders = len(sales_data)
    
    average_order_value = total_revenue / number_of_orders
    
    if total_revenue != 0:
        profit_margin = (total_profit / total_revenue) * 100
    else:
        profit_margin = 0
    
    return {
        "Total Revenue": total_revenue,
        "Total Profit": total_profit,
        "Total Quantity Sold": total_quantity,
        "Average Order Value": average_order_value,
        "Profit Margin (%)": profit_margin
    }

calculate_revenue_metrics(data)

{'Total Revenue': 2297200.8603,
 'Total Profit': 286397.0217,
 'Total Quantity Sold': 37873,
 'Average Order Value': 229.85800083049827,
 'Profit Margin (%)': 12.467217240315605}